In [1]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
import datetime
%matplotlib inline

In [2]:
def div(x,y):
    return 0 if y==0 else x/y

def gap_analyse(ticker, date, gap_perc_size):
    data = web.DataReader(ticker, data_source='yahoo', start=date)
    data['Last Close'] = data['Close'].shift()
    del data['Close']
    del data['Adj Close']
    del data['Volume']
    data['gap size'] = (data['Open']-data['Last Close'])
    
    
    up_gap = data[data['gap size']*100 / data['Last Close'] > gap_perc_size]
    up_gap = up_gap.reset_index(drop=True)
    up_gap['return'] = (up_gap['Open']- up_gap['Low'])/up_gap['gap size']
    up_gap['return'] = np.clip(up_gap['return'], a_max=gap_perc_size, a_min=None)
    
    down_gap = data[data['gap size']*100 / data['Last Close'] < (-1)*gap_perc_size]
    down_gap = down_gap.reset_index(drop=True)
    down_gap['return'] = (down_gap['Open']- down_gap['High'])/down_gap['gap size']
    down_gap['return'] = np.clip(down_gap['return'], a_max=gap_perc_size, a_min=None)
    

    res = {'Gap Ups': [round(up_gap['return'].mean(), 2), round(up_gap['return'].std(), 2), 
                       round(div(len(up_gap[up_gap['return']>=1])*100,len(up_gap)), 2), int(len(up_gap))],
          'Gap Downs': [round(down_gap['return'].mean(), 2), round(down_gap['return'].std(), 2), 
                        round(div(len(down_gap[down_gap['return']>=1])*100, len(down_gap)),2), int(len(down_gap))]}
    
    results = pd.DataFrame(data=res)
    results = results.rename(index = {0: 'Average', 1: '+/-', 2:'% close', 3:'Number of Gaps'})
    
    return(results)    

In [11]:
gap_analyse('AMC', '2021-01-26', 1)

,Gap Ups,Gap Downs
Average,0.69,0.37
+/-,0.27,0.29
% close,33.33,0.00
Number of Gaps,3.00,2.00


In [14]:
gap_analyse('TSLA', '2017-05-01', 1)

,Gap Ups,Gap Downs
Average,0.71,0.70
+/-,0.33,0.34
% close,39.41,43.58
Number of Gaps,236.00,179.00


In [5]:
gap_analyse('HOG', '2016-01-26', 10)

,Gap Ups,Gap Downs
Average,0.02,0.46
+/-,NaN,0.16
% close,0.00,0.00
Number of Gaps,1.00,3.00


In [3]:
def div(x,y):
    return 0 if y==0 else x/y

def test_gap_analyse(ticker, date, gap_perc_size):
    data = web.DataReader(ticker, data_source='yahoo', start=date)
    data['Last Close'] = data['Close'].shift()
    del data['Close']
    del data['Adj Close']
    del data['Volume']
    data['gap size'] = (data['Open']-data['Last Close'])
    
    
    up_gap = data[data['gap size']*100 / data['Last Close'] > gap_perc_size]
    up_gap = up_gap.reset_index(drop=True)
    up_gap['return'] = (up_gap['Open']- up_gap['Low'])/up_gap['gap size']
    up_gap['return'] = np.clip(up_gap['return'], a_max=gap_perc_size, a_min=None)
    
    down_gap = data[data['gap size']*100 / data['Last Close'] < (-1)*gap_perc_size]
    down_gap = down_gap.reset_index(drop=True)
    down_gap['return'] = (down_gap['Open']- down_gap['High'])/down_gap['gap size']
    down_gap['return'] = np.clip(down_gap['return'], a_max=gap_perc_size, a_min=None)

    
    res = {'U Ave': [round(up_gap['return'].mean(), 2)], 'U +-': [round(up_gap['return'].std(), 2)],
           'U % Close': [round(div(len(up_gap[up_gap['return']>=1])*100,len(up_gap)))], 'No U': [len(up_gap)],
           'D Ave': [round(down_gap['return'].mean(), 2)], 'D +-': [round(down_gap['return'].std(), 2)],
           'D % Close': [round(div(len(down_gap[down_gap['return']>=1])*100, len(down_gap)), 2)], 'No D': [len(down_gap)]}
    
    results = pd.DataFrame(data=res)
    results = results.rename(index = {0: str(ticker)})
    
    return(results) 

In [23]:
test_gap_analyse('AMC', '2021-02-01', 1)

KeyboardInterrupt: 

In [13]:
test_gap_analyse('TSLA', '2017-05-01', 1)

,GU Ave,GU +-,GU % Close,No GU,GD Ave,GD +-,GD % Close,No GD
ticker,0.71,0.33,39,236,0.7,0.34,43.58,179


In [4]:
test_gap_analyse('HOG', '2016-01-26', 10)

,U Ave,U +-,U % Close,No U,D Ave,D +-,D % Close,No D
HOG,0.02,NaN,0,1,0.46,0.16,0.0,3
